In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(r"customer_shopping_behavior.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

Imputing missing values with the category wise median

In [ ]:
df['Review Rating']= df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [ ]:
df.isnull().sum()

In [ ]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ","_")
df.columns

Renamimg columns

In [ ]:
df = df.rename(columns= {"purchase_amount_(usd)":"purchase_amt"})
df.columns

Defining Age groups

In [ ]:
labels = ['Young','Adult', 'Middle-aged','Senior']
df['age_group']= pd.qcut(df['age'], q=4, labels =labels)

In [ ]:
df[['age','age_group']].head()

Frequency mapping on the basis of days

In [ ]:
frequency_mapping = {'Fortnightly':14,
                    'Weekly':7,
                    'Monthly':30,
                    'Quarterly':90,
                    'Bi-Weekly':14,
                    'Annually':365,
                    'Every 3 Months':90
}
df['purchase_frequency_days']=df['frequency_of_purchases'].map(frequency_mapping)
df['purchase_frequency_days'].unique()

In [ ]:
df[['purchase_frequency_days']].head()

Removing redundant columns

In [ ]:
(df['discount_applied']==df['promo_code_used']).all()

In [ ]:
df.drop('promo_code_used',axis=1)

Connecting to MySQL

In [ ]:
!pip install mysql-connector-python
!pip install pandas sqlalchemy pymysql

In [ ]:
import os
import mysql.connector
from sqlalchemy import create_engine
import pymysql

In [ ]:
import sys
!{sys.executable} -m pip install pymysql


In [ ]:
 # Database credentials
db_user = "user"
db_pass = "password"        
db_host = "localhost"
db_name = "customer"

try:
    # Direct MySQL connection (mysql.connector)
    mydb = mysql.connector.connect(
        host=db_host,
        user=db_user,
        passwd=db_pass,
        database=db_name,
        use_pure=True
    )
    print("Connection successful!")


except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if 'mydb' in locals() and mydb.is_connected():
        mydb.close()
        print("Connection closed.")

#create engine
try:
    engine = create_engine(f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}")
    print("SQLAlchemy engine created successfully!")
except Exception as e:
    print(f"Error creating engine: {e}")

In [ ]:
df = df.rename(columns={"purchase_amount_(usd)": "purchase_amt"})
df.to_sql("customer", con=engine, if_exists="append", index=False)

In [ ]:
table_name = 'customer'
df.to_sql(table_name, con =engine, if_exists ='append', index=False)

In [ ]:
pd.read_sql('SELECT*FROM customer LIMIT 5;', engine)

In [ ]:
df.to_csv('finalbehaviour.csv', index=False)

In [ ]:
    from IPython.display import FileLink



    df.to_csv("processed_data.csv", index=False, sep=",", encoding="utf-8")

    FileLink("processed_data.csv")